In [3]:
import os
import numpy as np
import pandas as pd
import logging
from tqdm import tqdm
from datetime import datetime
import time

In [4]:
datapath = './数据/股评_HS300_with_year/'
textpath = './数据/股评_HS300_handled/text/'
usrpath = './数据/股评_HS300_handled/username/'
dtpath = './数据/股评_HS300_handled/update_time/'
def get_list(datapath, outpath):
    # 获取待处理的list
    try:
        datalist = os.listdir(datapath)
        donelist = os.listdir(outpath)
        tohandle_list = [x for x in datalist if x not in donelist]
        return tohandle_list
    except:
        return False

In [5]:
tohandle_list = get_list(datapath, textpath)

In [6]:
tohandle_list1 = tohandle_list[:2]
tohandle_list2 = tohandle_list[3:]
tohandle_list = tohandle_list1 + tohandle_list2

In [7]:
tohandle_list

['000333.csv', '000413.csv', '000629.csv', '000671.csv']

In [8]:
def text_preprocess(tohandle):
    datapath = './数据/股评_HS300_with_year/'
    textpath = './数据/股评_HS300_handled/text/'
    usrpath = './数据/股评_HS300_handled/username/'
    dtpath = './数据/股评_HS300_handled/update_time/'
    try:
        df = pd.read_csv(os.path.join(datapath,tohandle))
        # 处理时间
        df["update_time"] = df["update_time"].apply(lambda x: '20'+ x[5:] if len(x) > 16 else x)
        df["update_time"] = df["update_time"].apply(lambda x: 'd'+ x if int(x[2:4]) > 23 else x)
        df = df[~ df["update_time"].str.contains("d")]
        # 将标题和内容合并
        df["title"].fillna('', inplace=True)
        df["content"].fillna('', inplace=True)
        df["text"] = df["title"].str.cat(df["content"])
        df = df[["username", "update_time", "text"]].copy()
        # 删除df["cont"]里美元符号及之间的内容
        df["text"] = df["text"].str.replace(r'\$.*?\$', '', regex=True)
        df = df[~ df["text"].str.contains("东方财富APP")]
        df = df[df["text"].str.len() > 2]
        # 保存
        text_df = pd.DataFrame(df["text"])
        text_df.to_csv(os.path.join(textpath,tohandle),index=False, encoding ="utf-8-sig")

        urs_df = pd.DataFrame(df["username"])
        urs_df.to_csv(os.path.join(usrpath,tohandle),index=False)
        dt_df = pd.DataFrame(df["update_time"])
        dt_df.to_csv(os.path.join(dtpath,tohandle),index=False)
        del df, text_df, urs_df, dt_df
    except:
        return False


In [9]:

if tohandle_list != False or tohandle_list != []:
    for i in tqdm(range(len(tohandle_list))):
        text_preprocess(tohandle_list[i])
    print("处理完成")

100%|██████████| 4/4 [00:10<00:00,  2.50s/it]

处理完成
